In [ ]:
# Excersise 1,2,3,4

In [ ]:
import socket

server_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_socket.bind(('localhost', 65433))
print("UDP Server is ready to receive API data...")
def decipher_caesarMsg(message, key):
    new_message = []
    for letter in message:
        character = ord(letter)
        character = character - key
        new_message.append(chr(character))
    return "".join(new_message)



while True:
    #receive data from client
    data, client_address = server_socket.recvfrom(2048) #max 2048 bytes
    message = data.decode('utf-8')  #from bytes to string
    print(f"Received data from {client_address}: {data.decode()}")    

    try:
        user_info, encrypted_message, key_str = message.split(":")
        key = int(key_str) 
        decrypted_message = decipher_caesarMsg(encrypted_message.strip(), key) #decrypt msg encrypted by casesar cipher
        print(f"Decrypted message from {user_info.strip()}: {decrypted_message}")
    except ValueError:
        print("Invalid fomrat")

UDP Server is ready to receive API data...


In [ ]:
# Client

In [ ]:
import socket
import re

def validate_password(passcode):
    warnings = []
    if len(passcode) < 8:
        warnings.append("Passcode must be at least 8 characters long.")
    if not re.search(r'[~></?\!@#$,%]', passcode):
        warnings.append("Passcode must include at least one special character.")
    if not re.search(r'[A-Z]', passcode):
        warnings.append("Passcode must have at least one uppercase letter.")
    if not re.search(r'[a-z]', passcode):
        warnings.append("Passcode must have at least one lowercase letter.")
    return warnings

def encrypt_text(text, shift_value):
    encoded_text = []
    for char in text:
        encoded_char = ord(char) + shift_value
        encoded_text.append(chr(encoded_char))
    return "".join(encoded_text)

# Initialize UDP client socket
udp_socket = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
server_info = ('localhost', 65433)

# User authentication process
username = input("Enter your username: ").lower()

while True:
    passcode = input("Set your passcode: ")
    validation_issues = validate_password(passcode)
    
    if validation_issues:
        for warning in validation_issues:
            print(warning)
    else:
        break

user_info = {
    "username": username,
    "local_ip": socket.gethostbyname(socket.gethostname()),
    "passcode": passcode
}

# Message sending loop
while True:
    try:
        shift_key = int(input("Enter a shift key for encryption: "))
    except ValueError:
        print("Invalid input! Shift key must be a number.")
        continue

    user_message = input("Type your message: ")
    encoded_message = encrypt_text(user_message, shift_key)

    final_message = f"{user_info['username']} ({user_info['local_ip']}): {encoded_message}:{shift_key}"

    # Sending encrypted message to server
    udp_socket.sendto(final_message.encode('utf-8'), server_info)
    print("Encrypted message successfully sent!")

    if user_message.lower() == "exit":
        print("Closing chat session...")
        break

udp_socket.close()
